In [2]:
import numpy as np
import pandas as pd
import json
df = pd.read_csv("2025_Problem_C_Data/summerOly_athletes.csv")
count = {}
for i in range(df.shape[0]):
    row = df.iloc[i]
    noc = row['NOC']
    year = row['Year']
    year = int(year)
    if noc in count:
        if year in count[noc]:
            count[noc][year] += 1
        else:
            count[noc][year] = 1
    else:
        count[noc] = []
        count[noc][year] = 1
#
# with open("athlete_count.json", "w", encoding="utf-8") as json_file:
#     json.dump(count, json_file, indent=4)

In [18]:
import pandas as pd

# 假设数据存储在 'data.csv' 文件中
# 读取CSV文件
df = pd.read_csv("2025_Problem_C_Data/summerOly_athletes.csv")

# 添加一列用于统计奖牌数量
df['Medal'] = df['Medal'].fillna('No medal')

# 统计每个国家每年各类奖牌的数量
medal_counts = df.groupby(['NOC', 'Year', 'Medal']).size().unstack(fill_value=0)

# 重命名列
medal_counts.columns.name = None
medal_counts = medal_counts.rename(columns={
    'Gold': 'Gold',
    'Silver': 'Silver',
    'Bronze': 'Bronze',
    'No medal': 'No medal'
})

# 计算每个国家每年参与的运动员人数
participants = df.groupby(['NOC', 'Year']).size().reset_index(name='Participants')

# 计算大项和小项的数量
events_counts = df.groupby(['NOC', 'Year', 'Sport']).size().reset_index(name='Events')
events_counts = events_counts.groupby(['NOC', 'Year']).size().reset_index(name='Unique Sports')

# 合并数据
final_df = medal_counts.reset_index().merge(participants, on=['NOC', 'Year'], how='left')
final_df = final_df.merge(events_counts, on=['NOC', 'Year'], how='left')

# 重命名列
final_df = final_df.rename(columns={'Unique Sports': 'Events'})

# 填充 NaN 值为 0
final_df.fillna(0, inplace=True)

# 输出结果到CSV文件
final_df.to_csv('final_output.csv', index=False)

# 打印最终的 DataFrame
final_df

      NOC  Year  Bronze  Gold  No medal  Silver  Participants  Events
0     AFG  1936       0     0        16       0            16       2
1     AFG  1948       0     0        25       0            25       2
2     AFG  1956       0     0        12       0            12       1
3     AFG  1960       0     0        16       0            16       2
4     AFG  1964       0     0         8       0             8       1
...   ...   ...     ...   ...       ...     ...           ...     ...
3217  ZIM  2008       0     1        12       3            16       6
3218  ZIM  2012       0     0         9       0             9       4
3219  ZIM  2016       0     0        31       0            31       7
3220  ZIM  2020       0     0         5       0             5       4
3221  ZIM  2024       0     0         7       0             7       3

[3222 rows x 8 columns]


In [21]:
import pandas as pd
import pycountry

# 东道主数据
host_data = {
    1896: "Greece",
    1900: "France",
    1904: "United States",
    1908: "Great Britain",
    1912: "Sweden",
    1920: "Belgium",
    1924: "France",
    1928: "Netherlands",
    1932: "United States",
    1936: "Germany",
    1948: "Great Britain",
    1952: "Finland",
    1956: "Australia",
    1960: "Italy",
    1964: "Japan",
    1968: "Mexico",
    1972: "West Germany",
    1976: "Canada",
    1980: "Soviet Union",
    1984: "United States",
    1988: "South Korea",
    1992: "Spain",
    1996: "United States",
    2000: "Australia",
    2004: "Greece",
    2008: "China",
    2012: "Great Britain",
    2016: "Brazil",
    2020: "Japan",
    2024: "France",
    2028: "United States",
    2032: "Australia"
}

# 创建东道主的ISO Alpha-3编码字典
host_iso = {}
for year, country in host_data.items():
    try:
        if  country == "Great Britain":
            iso_code = "GBR"
        elif country == "Soviet Union":
            iso_code = "URS"
        # 获取国家的ISO Alpha-3编码
        elif country == "West Germany":
            iso_code = "FRG"
        else:
            iso_code = pycountry.countries.lookup(country).alpha_3
            host_iso[year] = iso_code
    except LookupError:
        # 处理查找不到国家的情况
        print(f"Country not found: {country}")

# 假设 final_df 是之前的汇总 DataFrame
# 现在添加 is_host 列
final_df['is_host'] = final_df.apply(
    lambda row: 1 if host_iso.get(row['Year']) == row['NOC'] else 0, axis=1
)

# 输出结果到CSV文件
final_df.to_csv('final_output_with_hosts.csv', index=False)

# 打印最终的 DataFrame
# print(final_df)

In [27]:
import pandas as pd

# 读取现有的CSV文件
df = pd.read_csv('final_output_with_hosts.csv')

# 创建年份范围
years = list(range(1896, 2025, 4))

# 创建一个新的 DataFrame 用于存储补全后的数据
complete_data = []

# 获取所有国家的 NOC 列表
nocs = df['NOC'].unique()

# 为每个国家补全年份记录
for noc in nocs:
    for year in years:
        # 检查该国家在该年份是否有记录
        if ((df['NOC'] == noc) & (df['Year'] == year)).any():
            # 如果有记录，则获取该记录
            record = df[(df['NOC'] == noc) & (df['Year'] == year)].iloc[0]
            complete_data.append(record)
        else:
            # 如果没有记录，则创建一行全为0的记录
            complete_data.append([noc, year, 0, 0, 0, 0, 0, 0, 0])

# 创建新的 DataFrame
complete_df = pd.DataFrame(complete_data, columns=df.columns)

# 输出结果到新的CSV文件


# 打印最终的 DataFrame
print(complete_df)

      NOC  Year  Bronze  Gold  No medal  Silver  Participants  Events  is_host
0     AFG  1896       0     0         0       0             0       0        0
1     AFG  1900       0     0         0       0             0       0        0
2     AFG  1904       0     0         0       0             0       0        0
3     AFG  1908       0     0         0       0             0       0        0
4     AFG  1912       0     0         0       0             0       0        0
...   ...   ...     ...   ...       ...     ...           ...     ...      ...
7717  ZIM  2008       0     1        12       3            16       6        0
7718  ZIM  2012       0     0         9       0             9       4        0
7719  ZIM  2016       0     0        31       0            31       7        0
7720  ZIM  2020       0     0         5       0             5       4        0
7721  ZIM  2024       0     0         7       0             7       3        0

[7722 rows x 9 columns]


In [28]:
complete_df = complete_df.drop(columns=['No medal'])
complete_df['total'] = complete_df['Gold'] + complete_df['Silver'] + complete_df['Bronze']
complete_df.to_csv('complete_output.csv', index=False)

In [4]:
count =  {key: count[key] for key in sorted(count)}
for key in count:
    count[key] = {key: count[key] for key in sorted(count)}


In [6]:
sorted_inner_data = {country: dict(sorted(years.items())) for country, years in count.items()}

In [9]:
years_range = range(1896, 2025, 4)

In [11]:
complete_data = {}
for country, years in sorted_inner_data.items():
    complete_years = {year: years.get(year, 0) for year in years_range}
    complete_data[country] = complete_years

In [13]:
complete_data = pd.DataFrame(complete_data)

In [16]:
column_sums = complete_data.sum()

# 检查每列是否有0
has_zero = (column_sums == 0).any()
# 输出结果
print("每列的和：")
print(column_sums)
print("\n有列的和为0:", has_zero)

每列的和：
AFG     138
AHO      74
AIN      46
ALB      82
ALG     639
       ... 
YEM      42
YMD       5
YUG    2046
ZAM     246
ZIM     321
Length: 234, dtype: int64

有列的和为0: True


In [69]:
import pandas as pd

# 读取原始数据
data = pd.read_csv('2025_Problem_C_Data/summerOly_athletes.csv')

# 创建一个新的DataFrame用于汇总
summary = data.groupby(['NOC', 'Year', 'Sport', 'Event']).agg(
    Gold=('Medal', lambda x: (x == 'Gold').sum()),
    Silver=('Medal', lambda x: (x == 'Silver').sum()),
    Bronze=('Medal', lambda x: (x == 'Bronze').sum())
).reset_index()

# 计算每个国家每年获得的奖牌总数
medal_summary = summary.groupby(['NOC', 'Year']).agg(
    Gold=('Gold', 'sum'),
    Silver=('Silver', 'sum'),
    Bronze=('Bronze', 'sum'),
    Participants=('Event', 'nunique'),
    Sports=('Sport', 'nunique')
).reset_index()

# 保存结果到新的CSV文件
medal_summary.to_csv('summary.csv', index=False)

KeyboardInterrupt: 

In [35]:
import csv
import json

# 读取CSV文件并建立 Team 到 NOC 的映射
team_to_noc = {}

with open("2025_Problem_C_Data/summerOly_athletes.csv", newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for entry in reader:
        team = entry['Team']
        noc = entry['NOC']

        # 只在团队还未被映射的情况下进行映射
        if team not in team_to_noc:
            team_to_noc[team] = noc

# 将映射保存到JSON文件
with open('mapping.json', 'w') as jsonfile:
    json.dump(team_to_noc, jsonfile, ensure_ascii=False, indent=4)

print("Team to NOC mapping has been saved to mapping.json successfully.")

Team to NOC mapping has been saved to mapping.json successfully.


In [60]:
import pandas as pd
df = pd.read_csv('2025_Problem_C_Data/complete_output.csv')
medal_table = pd.read_csv('2025_Problem_C_Data/summerOly_medal_counts.csv')
with open('mapping.json', 'r', encoding='utf-8') as jsonfile:
    team_to_noc = json.load(jsonfile)

In [61]:
# 要去除的 NOC 值
nocs_to_remove = ['ROC', 'British West Indies']

# 找出无法转换的 NOC 值并去除指定的值
unmapped_nocs = medal_table[~medal_table['NOC'].isin(team_to_noc.keys()) & ~medal_table['NOC'].isin(nocs_to_remove)]['NOC']

# 去重并转换为列表
unique_unmapped_nocs = unmapped_nocs.unique().tolist()

# 打印无法转换的值
print("无法转换的 NOC 值（去重后，排除 ROC 和 British West Indies）：", unique_unmapped_nocs)

无法转换的 NOC 值（去重后，排除 ROC 和 British West Indies）： ['Mixed team', 'Russian Empire', 'Ceylon', 'United Team of Germany', 'Taiwan', 'FR Yugoslavia']


In [63]:
# 无法转换的值及其对应的替换值
unmapped_values = ['Mixed team', 'Russian Empire', 'Ceylon', 'United Team of Germany', 'Taiwan', 'FR Yugoslavia']
replacement_values = ['MIX', 'RUS', 'SRI', 'GER', 'TPE', 'YUG']

# 创建映射字典
replacement_dict = dict(zip(unmapped_values, replacement_values))

# 替换 NOC 列中的值
medal_table['NOC'] = medal_table['NOC'].replace(replacement_dict)

# 打印转换后的 medal_table

In [62]:
# 使用映射替换 NOC 列
medal_table['NOC'] = medal_table['NOC'].map(team_to_noc).fillna(medal_table['NOC'])

# 打印转换后的 medal_table
print(medal_table)

      Rank  NOC  Gold  Silver  Bronze  Total  Year
0        1  USA    11       7       2     20  1896
1        2  GRE    10      18      19     47  1896
2        3  GER     6       5       2     13  1896
3        4  FRA     5       4       2     11  1896
4        5  GBR     2       3       2      7  1896
...    ...  ...   ...     ...     ...    ...   ...
1426    84  PER     0       0       1      1  2024
1427    84  QAT     0       0       1      1  2024
1428    84  SGP     0       0       1      1  2024
1429    84  SVK     0       0       1      1  2024
1430    84  ZAM     0       0       1      1  2024

[1431 rows x 7 columns]


In [66]:

# 找出所有不是三位数的 NOC 值
non_three_digit_nocs = medal_table[medal_table['NOC'].str.len() != 3]['NOC']

# 输出结果
if non_three_digit_nocs.empty:
    print("所有 NOC 值都是三位数。")
else:
    print("以下 NOC 值不是三位数：")
    print(non_three_digit_nocs.tolist())

以下 NOC 值不是三位数：
['British West Indies']


In [68]:
complete_data = pd.read_csv('2025_Problem_C_Data/complete_output.csv')
merged_data = complete_data.merge(medal_table, on=['NOC', 'Year'], how='left', suffixes=('', '_new'))

# 用新的值替换旧的值，在没有新值的情况下保持原值
complete_data['Gold'] = merged_data['Gold_new'].combine_first(complete_data['Gold'])
complete_data['Silver'] = merged_data['Silver_new'].combine_first(complete_data['Silver'])
complete_data['Bronze'] = merged_data['Bronze_new'].combine_first(complete_data['Bronze'])
complete_data['Total'] = merged_data['Total_new'].combine_first(complete_data['Total'])

# 打印更新后的 complete_data
complete_data.to_csv('complete_data.csv', index=False)